In [77]:
import pandas as pd

In [78]:
df = pd.read_csv("spotify_millsongdata.csv")

In [79]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [80]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [81]:
df.shape

(57650, 4)

In [82]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [83]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [84]:
df.head(10)

,artist,song,text
0,Hanson,Lulla Belle,Lulla Belle won the big show \r\nGot her pict...
1,Pharrell Williams,Can I Have It Like That,"Ladies and gentlemen, \r\nYeszir, \r\nLet's ..."
2,Squeeze,First Thing Wrong,There's evil in goodness and goodness in bad ...
3,Ne-Yo,Genuine Only,We may struggle \r\nWe may even very well see...
4,Kirk Franklin,Wake Up,"[Lead:] \r\nHeavenly Father, its me once agai..."
5,Young Buck,All My Life,Now we can't lose fo' sho' \r\nYa'll niggaz w...
6,Westlife,I Cry,You said goodbye \r\nI fell apart \r\nI fell...
7,Flo-Rida,I Know You Got A Man,"[Chorus] \r\nI know you got a man, man, man ..."
8,Avril Lavigne,Waste Of Time,Time is valuable \r\nMine is useless \r\nNex...
9,Irving Berlin,Oh How That German Could Love!,[1st verse:] \r\nOnce I got stuck on a sweet ...


In [85]:
df['text'][0]

"Lulla Belle won the big show  \r\nGot her picture framed by the window  \r\nIn the hall where lovely girls go  \r\nLulla Belle goes, Lulla Belle no  \r\n  \r\nLulla Belle, no, no, no, no, no  \r\nLulla Belle, no, ooh, oh, oh, ooh  \r\n  \r\nBeauty queen that you are  \r\nPut you down for a place in the stars  \r\nTake your picture for all the world to see  \r\n  \r\nLulla Belle' s leaving the ball  \r\nNo more reason to stay  \r\nOr people to see  \r\nCos everyone here knows your name  \r\nFrom the light on your face  \r\nIn the frame that hangs  \r\nIn the hall where lovely girls go  \r\nLulla Belle goes, Lulla Belle no  \r\n  \r\nLulla Belle, no, no, no, no, no  \r\nLulla Belle, no, ooh, oh, oh, ooh\r\n\r\n"

In [86]:
df.shape

(5000, 3)

In [87]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [88]:
import nltk
nltk.download('punkt')

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rajgu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [89]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [91]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [92]:
similarity[0]

array([1.        , 0.00787227, 0.00785907, ..., 0.00132435, 0.        ,
       0.0598631 ])

In [93]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [98]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:5]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs


In [99]:
recommendation('In Your Eyes')

['When I Look Into Your Eyes',
 'The Heart Of Everything',
 'I Close My Eyes',
 'Close Your Eyes']

In [100]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))